In [3]:
from cookbook.interface import PluginInstanceRedisInterface
import os
import sys

import logging

logging.basicConfig()
logger = logging.getLogger(__name__)

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''
redis_channel = os.environ.get("REDIS_CHANNEL")

# Increase the recursion limit in order to properly serialize Complexes
recursion_limit = 100000
sys.setrecursionlimit(recursion_limit)

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)


In [ ]:
# Load in example structures

from nanome.api.structure import Complex
from nanome.util import Color
# Load Test Molecules
examples_folder = os.path.join(os.getcwd(), 'examples')
pdb_6nzt = os.path.join(examples_folder, '6nzt.pdb')
pdb_6nzv = os.path.join(examples_folder, '6nzv.pdb')

comp_1 = Complex.io.from_pdb(path=pdb_6nzt)
comp_1.name = "6NZT"
comp_2 = Complex.io.from_pdb(path=pdb_6nzv)
comp_2.name = "6NZV"
comp_list = [comp_1, comp_2]
# plugin_instance.update_structures_deep(comp_list)


In [4]:
ws = plugin_instance.request_workspace()
comp_list = ws.complexes

moving_comp = comp_list[0]
fixed_comp = comp_list[1]

Sending request_workspace Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel a68bb66b-b23e-4616-b465-157d66398b0a


In [21]:
# Run FPocket, and highlight the moving structure potential pockets

from rmsd_v2.fpocket_client import FPocketClient
import time

comp = moving_comp


def highlight_atom_list(comp, atom_list):
    for atom in comp.atoms:
            atom.selected = atom in atom_list
    plugin_instance.update_structures_deep([comp])

    
def cycle_pockets(comp):
    fpocket_client = FPocketClient()
    logger.info(f"Showing pockets for {comp.name}")
    pocket_sets = fpocket_client.get_pockets(comp)
    current_index = 0

    for pocket in pocket_sets:
        print(f"Highlighting pocket {current_index}")
        highlight_atom_list(comp, pocket)
        current_index = (current_index + 1) % len(pocket_sets)

    # Deselect all atoms
    for atom in comp.atoms:
        atom.selected = False
    plugin_instance.update_structures_deep([comp])


cycle_pockets(comp)


Highlighting pocket 0
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel a4760da8-1688-4253-a0fe-799de5203419
Highlighting pocket 1
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 2d648ecc-2bdf-457f-8cd6-9441e269ad2f
Highlighting pocket 2
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 868ab5dd-9f8a-4cdf-b6db-5595192fa722
Highlighting pocket 3
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 1daad01e-3454-4f0f-b2ac-7fa21b7989ab
Highlighting pocket 4
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 218af9cb-9b0c-4741-a51c-8276b1109a33
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Re

In [22]:
# Highlight the fixed binding pocket according to the ligand.
from rmsd_v2.RMSDV2 import RMSDV2
instance = RMSDV2()

comp = fixed_comp
ligand_name = 'L9P'

def highlight_binding_site(comp, ligand_name: str, on_or_off):
    residue = next(res for res in comp.residues if res.name == ligand_name)
    binding_site_atoms = instance.calculate_binding_site_atoms(comp, residue.atoms)
    for atom in binding_site_atoms:
        atom.selected = on_or_off
    plugin_instance.update_structures_deep([comp])

highlight_binding_site(comp, ligand_name, True)
highlight_binding_site(comp, ligand_name, False)


Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 00504fa6-a616-4343-8663-5ae49e2fe985
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel a7498666-a30f-4d9b-a959-5023985b51ba
